In [1]:
import pandas as pd
import re
df = pd.read_csv('data/usa_2018.csv')
df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7
0,0,63.3%,Bradley Byrne*,GOP,True,36.7%,Robert Kennedy,Dem,False
1,1,63.3%,Bradley Byrne*,GOP,True,36.7%,Robert Kennedy,Dem,False
2,2,61.5%,Martha Roby*,GOP,True,38.5%,Tabitha Isner,Dem,False
3,3,61.5%,Martha Roby*,GOP,True,38.5%,Tabitha Isner,Dem,False
4,4,63.9%,Mike Rogers*,GOP,True,36.1%,Mallory Hagan,Dem,False


In [2]:
winners = df[[df.columns[2], df.columns[3]]]
losers = df[[df.columns[6], df.columns[7]]]
losers.columns = ['candidate', 'party']
winners.columns = ['candidate', 'party']
all_candidates = pd.concat([winners, losers])
all_candidates
all_candidates['candidate'] = all_candidates['candidate'].astype(str)
all_candidates['party'] = all_candidates['party'].astype(str)
all_candidates = all_candidates.applymap(lambda x: x.strip() if isinstance(x, str) else x)
all_candidates.drop_duplicates(inplace = True, subset = ['candidate', 'party'])
p = re.compile(r'[^\w\s]+')
all_candidates['candidate'] = [p.sub('', x) for x in all_candidates['candidate'].tolist()]
all_candidates.head()

,candidate,party
0,Bradley Byrne,GOP
2,Martha Roby,GOP
4,Mike Rogers,GOP
6,Robert Aderholt,GOP
8,Mo Brooks,GOP


In [3]:
all_candidates['candidate'] = all_candidates['candidate']+" "+all_candidates['party']
search_terms = all_candidates['candidate']
search_terms


0             Bradley Byrne GOP
2               Martha Roby GOP
4               Mike Rogers GOP
6           Robert Aderholt GOP
8                 Mo Brooks GOP
10              Gary Palmer GOP
12                Don Young GOP
14            Tom OHalleran Dem
16          Ann Kirkpatrick Dem
18            Raul Grijalva Dem
20               Paul Gosar GOP
22               Andy Biggs GOP
24         David Schweikert GOP
26            Ruben Gallego Dem
28             Debbie Lesko GOP
30             Greg Stanton Dem
32            Rick Crawford GOP
34              French Hill GOP
36             Steve Womack GOP
38          Bruce Westerman GOP
40             Doug LaMalfa GOP
42            Jared Huffman Dem
44           John Garamendi Dem
46           Tom McClintock GOP
48            Mike Thompson Dem
50             Doris Matsui Dem
52                 Ami Bera Dem
54                Paul Cook GOP
56           Jerry McNerney Dem
58              Jeff Denham GOP
                 ...           
726     

In [4]:
from googleapiclient.discovery import build
import pprint
import pandas as pd
from PIL import Image
import requests
from io import BytesIO
import cv2
import numpy as np
import os
import time
import requests
from PIL import Image
from io import BytesIO
from keys_file import GOOGLE_KEY
def get_imgs_from_urls(img_urls):
    images = []
    for image_url in img_urls:
            print(image_url)
            try:
                response = requests.get(image_url)
                image = Image.open(BytesIO(response.content))
                images.append(image)
            except: 
                print("ERROR")
                None
    return images

def get_image_from_json(json):
    images = []
    for item in json['value']:
        try:
            response = requests.get(item['contentUrl'])
            image = Image.open(BytesIO(response.content))
            images.append(image)
        except: 
            None
    return images
def count_faces(image):
    xml_files = ['xml_files/haarcascade_frontalface_default.xml','xml_files/lbpcascade_frontalface.xml']
    faces_counted = []
    for xml_file in xml_files:
        face_cascade = cv2.CascadeClassifier(xml_file)
        gray = cv2.cvtColor(np.float32(image), cv2.COLOR_BGR2GRAY)
        gray = np.array(gray, dtype='uint8')
        faces = face_cascade.detectMultiScale(gray, 1.2, 5)
        faces_counted.append(len(faces))
        if len(faces) == 1:
            return 1
            break
    return 0
        
# MTCNN uses depricted functions.
def count_faces_dnn(image):
    img = cv2.cvtColor(np.float32(image), cv2.COLOR_BGR2RGB)
    detector = MTCNN()
    faces = detector.detect_faces(img)
    print(len(faces))    
    return None
        

def check_directory(query):
    directory = 'images usa 2018/'+query
    if not os.path.exists(directory):
        os.makedirs(directory)
    return directory

def select_relevant_images(images):
#   check if url contains portrait
    relevant_images = []
    for image  in images:
        faces_count = 0
        try:
            faces_count = count_faces(image)
        except:
            None
        if faces_count == 1:
            relevant_images.append(image)
    return relevant_images

def write_images_to_directory(directory, images, query):
    counter = 1
    for image in images:
        if not image.mode == 'RGB':
            image = image.convert('RGB')
        image.save(directory+"/google_"+query+str(counter)+".jpeg")
        counter += 1;
        
def get_google_urls(search_term):
    service = build("customsearch", "v1",
            developerKey= GOOGLE_KEY)

    res = service.cse().list(
      q=search_term,
      cx='014171572285050841134:z8jpb8kffjb',
      num='8',
      searchType='image',
      imgType='face'
    ).execute()
    urls = []
    for item in res['items']:
        urls.append(item['link'])
    return urls


In [6]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
counter = 0
for search_term in search_terms[168:]:
    print(search_term)
    print(counter, "of", len(search_terms[168:]))
    urls = get_google_urls(search_term)
    print("getting urls")
    if urls:
#         print(urls)
        print("GETTINF IMAGES")
        images = get_imgs_from_urls(urls)
        print("TO DIRECTORY")

        directory = check_directory(search_term)
#         images = select_relevant_images(images)
        write_images_to_directory(directory, images, search_term)
        print("DONE")
    else:
        print('No images found') 
    time.sleep(2)
    counter += 1
print("DONE")

Jamie Raskin Dem
0 of 618
getting urls
GETTINF IMAGES
https://s.yimg.com/ny/api/res/1.2/LsSKYdFr9XPdltBEJvYOZw--/YXBwaWQ9aGlnaGxhbmRlcjt3PTcwNTtoPTM5NS43ODk0NzM2ODQyMTA1/https://s.yimg.com/uu/api/res/1.2/tNCRGah5k6e_ARdrwmdHMQ--~B/aD04MzI7dz0xNDgyO3NtPTE7YXBwaWQ9eXRhY2h5b24-/https://media-mbst-pub-ue1.s3.amazonaws.com/creatr-uploaded-images/2019-12/2a9d6420-2421-11ea-af47-faea62c54175
https://bloximages.chicago2.vip.townnews.com/stardem.com/content/tncms/assets/v3/editorial/9/cc/9cc2e821-c73c-5eac-a65e-2a4ffa91b757/58de6917ce0d8.image.jpg?resize=400%2C267
https://thehill.com/sites/default/files/styles/article_full/public/raskinjamie_120116gn_lead.jpg?itok=IBDrYfkN
https://www.thenation.com/wp-content/uploads/2018/07/jamie-raskin-ap-img.jpg
https://s.yimg.com/ny/api/res/1.2/J.r0iKyaBQ_xGRi7SRxsmA--~A/YXBwaWQ9aGlnaGxhbmRlcjtzbT0xO3c9ODAw/https://media-mbst-pub-ue1.s3.amazonaws.com/creatr-uploaded-images/2019-12/2a9d6420-2421-11ea-af47-faea62c54175
https://www.newsmax.com/CMSPages/GetFile

https://www.wkar.org/sites/wkar/files/styles/x_large/public/201905/88FEB323-B4CA-4F7A-98A0-460E15C974B2.jpeg
https://twt-thumbs.washtimes.com/media/image/2018/03/01/beltway_amash_c0-0-1177-686_s885x516.jpg?f270ebcce5024fffb22ce98bfcd860cce9c41a05
TO DIRECTORY
DONE
John Moolenaar GOP
9 of 618
getting urls
GETTINF IMAGES
https://s.hdnux.com/photos/01/07/45/77/18767868/3/920x920.jpg
https://www.gannett-cdn.com/-mm-/113c8f06ae420de76505f464d6b72f2cce70a191/c=0-24-565-777/local/-/media/Lansing/2014/08/05/johnmoolenaar.jpg?width=534&height=712&fit=crop
https://www.gop.gov/wp-content/uploads/2015/01/Moolenaar.jpg
https://editorial01.shutterstock.com/wm-preview-1500/6083720t/b3c159a0/gop-2016-michigan-conference-mackinac-island-usa-shutterstock-editorial-6083720t.jpg
https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/200/300/John_Moolenaar.jpg
https://pbs.twimg.com/profile_images/997236401849622528/EYBv4I7c_400x400.jpg
https://lookaside.fbsbx.com/lookaside/crawler/media/?media_id=572656369

https://media1.fdncms.com/metrotimes/imager/u/original/2451166/dingell-pic_1.jpg
https://www.clickondetroit.com/resizer/UwXzsg12ehXEaH3BcGdJELy5o2k=/927x608/smart/filters:format(jpeg):strip_exif(true):strip_icc(true):no_upscale(true):quality(65)/arc-anglerfish-arc2-prod-gmg.s3.amazonaws.com/public/BRASZG2ZMJC5VKPS5ZJXTMJMSQ.png
https://www.govtrack.us/static/legislator-photos/412637-200px.jpeg
TO DIRECTORY
DONE
Rashida Tlaib Dem
18 of 618
getting urls
GETTINF IMAGES
https://a57.foxnews.com/media2.foxnews.com/BrightCove/694940094001/2019/03/07/931/524/694940094001_6011224070001_6011217118001-vs.jpg?ve=1&tl=1
https://images.jpost.com/image/upload/f_auto,fl_lossy/t_JD_ArticleMainImageFaceDetect/453505
https://media.breitbart.com/media/2019/08/Angry-Rashida-Tlaib-Confronts-Detroit-Police-640x480.jpg
https://a57.foxnews.com/cf-images.us-east-1.prod.boltdns.net/v1/static/694940094001/0a3287e8-0884-4d8e-9af0-fbf613a7b999/0b981612-9f88-451e-aacf-f3f193e7f3b4/1280x720/match/931/524/image.jpg?ve

https://stmedia.stimg.co/1453089241_10017641+Emmers+Return.JPG?fit=crop&crop=faces
https://www.twincities.com/wp-content/uploads/2015/11/20140803__140803_emmer_simirajah_skybox.jpg?w=599
https://minnesota.cbslocal.com/wp-content/uploads/sites/15909630/2010/11/tomemmer.jpg?w=640
https://stmedia.stimg.co/2emmer060613.jpg?fit=crop&crop=faces
TO DIRECTORY
DONE
Collin Peterson Dem
26 of 618
getting urls
GETTINF IMAGES
https://www.twincities.com/wp-content/uploads/2015/11/20130904__peterson-collin.jpg
https://upload.wikimedia.org/wikipedia/commons/f/fb/Collin_Peterson_official_photo.jpg
https://media3.s-nbcnews.com/j/newscms/2019_44/3078326/191031-collin-peterson-jeff-van-drew-al-1240_58618755e750a2a03ce1abe33ad5aa0d.fit-760w.jpg
https://media.breitbart.com/media/2019/01/Collin-Peterson-640x480.jpg
https://tennesseestar.com/wp-content/uploads/2019/12/Webb-challenges-Dems-to-reject-impeachment_840x480.jpg
https://img.apmcdn.org/a5639c5770b073a6d0c635ccfd4ac6e7f3289ac7/widescreen/32cb4c-201911

https://cdn.cnn.com/cnnnext/dam/assets/190205232741-rep-ann-wagner-080218-large-169.jpg
https://stcdems.org/wp-content/uploads/2017/06/ann-wagner.jpg
ERROR
https://bloximages.newyork1.vip.townnews.com/stltoday.com/content/tncms/assets/v3/editorial/f/72/f72f5a36-0304-11e0-b6a7-00127992bc8b/4cffe27f1d6c3.image.jpg
https://storage.googleapis.com/afs-prod/media/media:5ab3d62853504677a8b11eda27aa125a/800.jpeg
https://bloximages.newyork1.vip.townnews.com/stltoday.com/content/tncms/assets/v3/editorial/f/72/f72f5a36-0304-11e0-b6a7-00127992bc8b/4cffe27f2210c.preview-1024.jpg?crop=768%2C768%2C128%2C0&resize=1200%2C1200&order=crop%2Cresize
https://news.stlpublicradio.org/sites/kwmu/files/styles/x_large/public/201810/102318_JR_WagnerVanOstranCollage.jpg
TO DIRECTORY
DONE
Blaine Luetkemeyer GOP
34 of 618
getting urls
GETTINF IMAGES
https://thehill.com/sites/default/files/styles/thumb_small_article/public/luetkemeyerblaine_072017gn2_lead.jpg?itok=7XdpEifI
https://upload.wikimedia.org/wikipedia/commo

...

https://upload.wikimedia.org/wikipedia/commons/thumb/c/c6/Juli%C3%A1n_Castro%27s_Official_HUD_Portrait.jpg/1200px-Juli%C3%A1n_Castro%27s_Official_HUD_Portrait.jpg
https://bloximages.chicago2.vip.townnews.com/hjnews.com/content/tncms/assets/v3/editorial/9/75/975de6c8-790a-5491-ae77-7bacc3683d09/5ae4f8827b06e.image.jpg
https://upload.wikimedia.org/wikipedia/commons/c/c6/Juli%C3%A1n_Castro%27s_Official_HUD_Portrait.jpg
https://i.pinimg.com/originals/fe/14/56/fe1456d04b603d2c7026240f7bbde802.jpg
TO DIRECTORY
DONE
Shireen Ghorbani Dem
583 of 618
getting urls
GETTINF IMAGES
https://utahdemocrats.org/wp-content/uploads/2019/10/shireen-ghorbani.jpg
https://lookaside.fbsbx.com/lookaside/crawler/media/?media_id=10156576634363432
ERROR
https://pbs.twimg.com/profile_images/1073463791898116096/9W53s6wW_400x400.jpg
https://lookaside.fbsbx.com/lookaside/crawler/media/?media_id=130434487619538
ERROR
https://pbs.twimg.com/profile_images/1073463791898116096/9W53s6wW.jpg
https://i1.wp.com/www.utahpolitic

https://a57.foxnews.com/a57.foxnews.com/static.foxnews.com/foxnews.com/content/uploads/2018/09/640/320/1862/1048/Wilde20Cockburn.jpg?ve=1&tl=1?ve=1&tl=1
https://denverforcongress.com/wp-content/uploads/2018/10/denver-riggleman-leslie-cockburn-640x480.jpg
TO DIRECTORY
DONE
Jennifer Lewis Dem
591 of 618
getting urls
GETTINF IMAGES
https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/200/300/27993232_149552352399444_4114351601589374745_o.jpg
https://c8.alamy.com/comp/FNFFHC/jennifer-lewis-bei-der-prsentation-der-tv-serie-black-ish-auf-dem-FNFFHC.jpg
https://pbs.twimg.com/profile_images/746821017423613952/kWJX8GGK.jpg
https://www.wmra.org/sites/wmra/files/styles/medium/public/201805/dem_jenniferlewis.jpeg
https://pbs.twimg.com/profile_images/746821017423613952/kWJX8GGK_400x400.jpg
https://upload.wikimedia.org/wikipedia/commons/3/35/Jenifer_Lewis.jpg
https://i.ytimg.com/vi/J6Rr3WuG8FE/maxresdefault.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/3/35/Jenifer_Lewis.jpg/220px-Jenif

https://bloximages.chicago2.vip.townnews.com/chronline.com/content/tncms/assets/v3/editorial/e/25/e25ec40e-a1b9-11e9-81dc-8f67a5acf595/5d239eef0bf0d.image.jpg?resize=1200%2C800
https://static.seattletimes.com/wp-content/uploads/2018/10/dems-1560x843.jpg
https://images.axios.com/KjtODpOULOgiDGrcA5JpZQe6nww=/1920x1080/smart/2017/12/15/1513306984657.jpg
https://static.seattletimes.com/wp-content/uploads/2018/10/dems-1200x630.jpg
TO DIRECTORY
DONE
Christine Brown Dem
600 of 618
getting urls
GETTINF IMAGES
https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/200/300/Christine_Brown.jpg
https://media.graytvinc.com/images/810*455/1280x720_80814P00-MKJVQ.jpg
https://www.nydailynews.com/resizer/rW31MFUcjljXXZpZTCHDIlRIpF4=/415x311/top/arc-anglerfish-arc2-prod-tronc.s3.amazonaws.com/public/XCRNPYHF7SKMV4GJLNK5454O7Y.jpg
ERROR
https://s.yimg.com/ny/api/res/1.2/sQSxl0KElqXX5v3QiR53ag--~A/YXBwaWQ9aGlnaGxhbmRlcjtzbT0xO3c9NjIwO2g9NDE0/http://media.zenfs.com/en-us/homerun/deadline.com/3e158a490c6811

https://scottrasmussen.com/wp-content/uploads/KFershee-min-200x300.jpg
https://s.abcnews.com/images/Politics/kyle-horton-kendra-fershee-liz-watson-split-ht-ap-jc-180509_hpMain_4x5_992.jpg
http://media.graytvinc.com/images/Alex+Mooney+vs+Sergent.jpg
https://s.abcnews.com/images/Politics/kyle-horton-kendra-fershee-liz-watson-split-ht-ap-jc-180509_hpMain_1x1_608.jpg
https://upload.wikimedia.org/wikipedia/commons/a/a5/Official_photo_of_Speaker_Nancy_Pelosi_in_2019.jpg
TO DIRECTORY
DONE
Talley Sergent Dem
608 of 618
getting urls
GETTINF IMAGES
https://i.ytimg.com/vi/rf0XpA7AGOE/maxresdefault.jpg
https://pbs.twimg.com/profile_images/1062824925206138880/7ipxq8Of_400x400.jpg
http://media.graytvinc.com/images/Alex+Mooney+vs+Sergent.jpg
https://pbs.twimg.com/profile_images/1062824925206138880/7ipxq8Of.jpg
https://upload.wikimedia.org/wikipedia/commons/a/a5/Official_photo_of_Speaker_Nancy_Pelosi_in_2019.jpg
https://bloximages.newyork1.vip.townnews.com/williamsondailynews.com/content/tncms/assets/

https://images.dailykos.com/images/506585/story_image/Margaret_Head_Shot-Square.jpg?1518369861
http://www.crispreport.com/wp-content/uploads/2018/08/1534300941465.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/a/a5/Official_photo_of_Speaker_Nancy_Pelosi_in_2019.jpg/1200px-Official_photo_of_Speaker_Nancy_Pelosi_in_2019.jpg
https://bluewisconews.files.wordpress.com/2018/03/engebretson-e1520466836314.jpg?w=301&h=312
TO DIRECTORY
DONE
Beau Liegeois Dem
616 of 618
getting urls
GETTINF IMAGES
http://indivisibledoorcounty.com/wp-content/uploads/2017/09/img003-3.jpg
https://i.ytimg.com/vi/AMxg1a5ue14/maxresdefault.jpg
http://indivisibledoorcounty.com/wp-content/uploads/2017/09/img003-3-1600x2107.jpg
https://upload.wikimedia.org/wikipedia/commons/a/a5/Official_photo_of_Speaker_Nancy_Pelosi_in_2019.jpg
https://bloximages.chicago2.vip.townnews.com/journaltimes.com/content/tncms/assets/v3/editorial/4/45/4459f1f4-e92f-53d1-8d34-7ab6078ad6c5/5d4b0aa34d024.image.jpg?crop=967%2C1182%2C448%2C

In [7]:
all_candidates.to_csv('data/all_candidates_usa_2018.csv')
# 137 of 348
len(all_candidates)

786